In [321]:
!jupytext --to py Character_Level_RNN_Exercise.ipynb

[jupytext] Reading Character_Level_RNN_Exercise.ipynb
[jupytext] Writing Character_Level_RNN_Exercise.py (destination file replaced)


# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
print(text[:101])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everything


### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
print(encoded[:101])

[38 71 16 62 20 18 70 19 75 58 58 58 66 16 62 62 28 19 29 16 74 72 46 72
 18 54 19 16 70 18 19 16 46 46 19 16 46 72 11 18 41 19 18 39 18 70 28 19
  1 63 71 16 62 62 28 19 29 16 74 72 46 28 19 72 54 19  1 63 71 16 62 62
 28 19 72 63 19 72 20 54 19 81 24 63 58 24 16 28 59 58 58 73 39 18 70 28
 20 71 72 63 60]


## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1, 7]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches.**

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches.** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches.**

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    # how many seq_length by batch_size fit into our array
    n_batches = len(arr)//((batch_size)*(seq_length))
    
    ## TODO: Keep only enough characters to make full batches
    # number of chars = n_batch*batch_size*seq_length
    arr = arr[:n_batches*batch_size*seq_length]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size,-1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:,n:n+seq_length]
        y = np.zeros_like(x)
        # The targets, shifted by one
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [130]:
def get_batches_overlapped(arr, batch_size, seq_len):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    N = len(arr)
    # if we can not reshape arr with axis0 to batch_size - we add elements from the beginning
    if N%batch_size != 0:
        N_to_add = batch_size - N%batch_size
        arr = np.append(arr, arr[:N_to_add])
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size,-1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1]-2, seq_len-1):
        if n < arr.shape[1] - 1:
            if n+seq_len >= arr.shape[1]:
                x = arr[:,-seq_len-1:-1]
                y = arr[:,-seq_len:]
            else:
                x = arr[:,n:n+seq_len]            
                y = arr[:,n+1:n+seq_len+1]
        yield x,y

In [104]:
arr = np.array(range(0,28))
batch_size = 2
seq_len = 3
for item in get_batches_overlapped(arr, batch_size, seq_len):
    for rowx, rowy in zip(item[0], item[1]):
        print(rowx, '\n', rowy)
    print()

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20 21 22 23 24 25 26 27]]
[0 1 2] 
 [1 2 3]
[14 15 16] 
 [15 16 17]

[2 3 4] 
 [3 4 5]
[16 17 18] 
 [17 18 19]

[4 5 6] 
 [5 6 7]
[18 19 20] 
 [19 20 21]

[6 7 8] 
 [7 8 9]
[20 21 22] 
 [21 22 23]

[ 8  9 10] 
 [ 9 10 11]
[22 23 24] 
 [23 24 25]

[10 11 12] 
 [11 12 13]
[24 25 26] 
 [25 26 27]



### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [105]:
encoded

array([38, 71, 16, ..., 54, 59, 58])

In [106]:
batches = get_batches_overlapped(encoded, 8, 50)
x, y = next(batches)

[[38 71 16 ... 54 81 63]
 [19 20 71 ... 19 81 70]
 [19 16 19 ... 76 71 72]
 ...
 [46 28 54 ... 20 71 16]
 [20 19 12 ... 18 28 82]
 [19 76 16 ... 59 58 38]]


In [107]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[38 71 16 62 20 18 70 19 75 58]
 [19 20 71 16 20 19 16 20 20 70]
 [19 16 19 29 81 18  6 19 71 18]
 [18 29 19 20 71 81  1 60 71 19]
 [70 25 54 20 16 72 63 18  5  6]
 [46 28 54 72 54  6 19 24 16 54]
 [20 19 12 81 46 46 28 19 24 81]
 [19 76 16 63 63 81 20 19 60 70]]

y
 [[71 16 62 20 18 70 19 75 58 58]
 [20 71 16 20 19 16 20 20 70 16]
 [16 19 29 81 18  6 19 71 18 19]
 [29 19 20 71 81  1 60 71 19 71]
 [25 54 20 16 72 63 18  5  6 19]
 [28 54 72 54  6 19 24 16 54 19]
 [19 12 81 46 46 28 19 24 81  1]
 [76 16 63 63 81 20 19 60 70 16]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [108]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [110]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super(CharRNN, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.n_chars = len(tokens)
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        # input_size – The number of expected features in the input x
        # hidden_size – The number of features in the hidden state h
        # num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking 
        # two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1
        # bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
        # batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False
        # dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0
        # bidirectional – If True, becomes a bidirectional LSTM. Default: False
        self.rnn = torch.nn.LSTM(input_size = self.n_chars, 
                                 hidden_size=self.n_hidden, 
                                 num_layers=self.n_layers, 
                                 batch_first=True, 
                                 dropout=self.drop_prob )
        
        # output of LSTM has a dim of hidden size and we then transform that into the number of chars we have and apply softmax
        self.fc = torch.nn.Linear(self.n_hidden, self.n_chars)
        
        self.dropout = torch.nn.Dropout(p=drop_prob)
    
    def num_parameters(self):
        '''
        get the number of parameters in a network
        '''
        
        # return sum((list(map(lambda x: torch.as_tensor(x.flatten().size()).sum().item(), self.parameters()))))
        s=""
        for k, v in self.named_parameters():
            s+=f'{k:20} {v.shape}\n'
        s+=f'Total number of parameters = {sum(list(map(lambda x: x.numel(), net.parameters())))}'
        return s
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. 
            hidden is a tuple of (h_n, c_n)
        '''
                
        ## TODO: Get the outputs and the new hidden state from LSTM
        # return the final output and the hidden(h_n, c_n) state
        out, hidden = self.rnn(x, hidden)
        # out dim = (batch, seq_len, num_directions * hidden_size)
        
        #print(out.shape, hidden[0].shape)
        # torch.Size([2, 50, 256]) (batch, seq_len, num_directions * hidden_size)
        # torch.Size([2, 2, 256]) (n_layers, batch_size, hidden_size)
        # this print shows that out and hidden are the same if you take the last cell from LSTM stack
        # print(out[:,-1:,:10],'\n', hidden[0][-1:,:,:10])
        
        out = self.dropout(out) # apply a dropout
        
        #out = out.contiguous().view(-1, self.n_hidden)
        # dim = (seq_len*batch, n_chars)
        
        out = self.fc(out)
        # dim = (seq_len, batch, n_chars)
        
        return out.view(-1, self.n_chars), hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new_zeros(self.n_layers, batch_size, self.n_hidden), weight.new_zeros(self.n_layers, batch_size, self.n_hidden))
        
        return hidden
    
    
#     def init_hidden(self, batch_size):
#         ''' Initializes hidden state '''
#         # Create two new tensors with sizes n_layers x batch_size x n_hidden,
#         # initialized to zero, for hidden state and cell state of LSTM
#         weight = next(self.parameters()).data
        
#         # (num_layers * num_directions, batch, hidden_size)
#         if (train_on_gpu):
#             hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
#                   weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
#         else:
#             hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
#                       weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
#         return hidden
        

In [111]:
# get the 1st batch
batches = get_batches(encoded, 2, 50)
x, y = next(batches)
print(x.shape)
x, y

(2, 50)


(array([[38, 71, 16, 62, 20, 18, 70, 19, 75, 58, 58, 58, 66, 16, 62, 62,
         28, 19, 29, 16, 74, 72, 46, 72, 18, 54, 19, 16, 70, 18, 19, 16,
         46, 46, 19, 16, 46, 72, 11, 18, 41, 19, 18, 39, 18, 70, 28, 19,
          1, 63],
        [19, 54, 16, 24, 19, 71, 18, 70, 19, 20, 18, 16, 70, 25, 54, 20,
         16, 72, 63, 18,  5,  6, 19, 62, 72, 20, 72, 29,  1, 46,  6, 19,
         54, 24, 18, 18, 20, 19, 29, 16, 76, 18,  6, 58, 74, 72, 54, 18,
         70, 16]]),
 array([[71, 16, 62, 20, 18, 70, 19, 75, 58, 58, 58, 66, 16, 62, 62, 28,
         19, 29, 16, 74, 72, 46, 72, 18, 54, 19, 16, 70, 18, 19, 16, 46,
         46, 19, 16, 46, 72, 11, 18, 41, 19, 18, 39, 18, 70, 28, 19,  1,
         63, 71],
        [54, 16, 24, 19, 71, 18, 70, 19, 20, 18, 16, 70, 25, 54, 20, 16,
         72, 63, 18,  5,  6, 19, 62, 72, 20, 72, 29,  1, 46,  6, 19, 54,
         24, 18, 18, 20, 19, 29, 16, 76, 18,  6, 58, 74, 72, 54, 18, 70,
         16, 21]]))

In [112]:
# one hot encode the data
x = one_hot_encode(x, len(chars))
print('Vocab size ', len(chars))
print(x.shape)

Vocab size  83
(2, 50, 83)


In [113]:
# the 1st element is 8 and we see the one on 9th position
x[0,1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [114]:
# create network and send it to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = 'cpu'
print(device)
net = CharRNN(chars).to(device)
net

cuda


CharRNN(
  (rnn): LSTM(83, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=83, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [115]:
# make an input tensor
tr_x = torch.FloatTensor(x).to(device)
tr_x[0,1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')

In [116]:
# initialize the hidden state
hidden = net.init_hidden(2)
hidden

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0'),
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0'))

In [117]:
hidden[0].requires_grad

False

In [118]:
out, h = net(tr_x,hidden)

In [119]:
out.shape, h[0].shape, h[1].shape

(torch.Size([100, 83]), torch.Size([2, 2, 256]), torch.Size([2, 2, 256]))

In [120]:
out[:1].sum()

tensor(-0.6064, device='cuda:0', grad_fn=<SumBackward0>)

In [122]:
print(net.num_parameters())

rnn.weight_ih_l0     torch.Size([1024, 83])
rnn.weight_hh_l0     torch.Size([1024, 256])
rnn.bias_ih_l0       torch.Size([1024])
rnn.bias_hh_l0       torch.Size([1024])
rnn.weight_ih_l1     torch.Size([1024, 256])
rnn.weight_hh_l1     torch.Size([1024, 256])
rnn.bias_ih_l1       torch.Size([1024])
rnn.bias_hh_l1       torch.Size([1024])
fc.weight            torch.Size([83, 256])
fc.bias              torch.Size([83])
Total number of parameters = 896851


## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [286]:
a = torch.ones(4)

In [288]:
a.requires_grad

False

In [289]:
b = a + 4

In [290]:
b.requires_grad

False

In [149]:
from tqdm.notebook import tqdm, trange

def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10, val_loss_min=np.inf):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    #opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, nesterov=True)
    #opt = torch.optim.RMSprop(net.parameters(), lr=lr, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=opt, T_max = 5*len(data)//(batch_size*seq_length), eta_min=1e-7)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
#     val_idx = int(len(data)*(1-val_frac))
#     data, val_data = data[:val_idx], data[val_idx:]
    val_idx = int(len(data)*(val_frac))
    val_data, data  = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    for e in trange(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # basically the same stuff
            # h = tuple([each.detach() for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            scheduler.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                with torch.no_grad(): 
                    for x, y in get_batches_overlapped(val_data, batch_size, seq_length):
                        # One-hot encode our data and make them Torch tensors
                        x = one_hot_encode(x, n_chars)
                        x, y = torch.from_numpy(x), torch.from_numpy(y)
                        inputs, targets = x, y
                        if(train_on_gpu):
                            inputs, targets = inputs.cuda(), targets.cuda()

                        # Creating new variables for the hidden state, otherwise
                        # we'd backprop through the entire training history
                        # val_h = tuple([each.data for each in val_h])       

                        output, val_h = net(inputs, val_h)
                        val_loss = criterion(output, targets.view(batch_size*seq_length))

                        val_losses.append(val_loss.item())
                        
                    if np.mean(val_losses) < val_loss_min:
                        print('Validation loss decreases, saving model parameters')
                        torch.save(net.state_dict(),'anna_model.pt')
                        val_loss_min = np.mean(val_losses)
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
    return val_loss_min

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [133]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden = 256
n_layers = 3

net = CharRNN(chars, n_hidden, n_layers, drop_prob=0.5)
print(net, f'\n\nNumber of parameters in network : {net.num_parameters()}', f'\n\nNumber of chars in text : {len(text):,}')


CharRNN(
  (rnn): LSTM(83, 256, num_layers=3, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=83, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
) 

Number of parameters in network : rnn.weight_ih_l0     torch.Size([1024, 83])
rnn.weight_hh_l0     torch.Size([1024, 256])
rnn.bias_ih_l0       torch.Size([1024])
rnn.bias_hh_l0       torch.Size([1024])
rnn.weight_ih_l1     torch.Size([1024, 256])
rnn.weight_hh_l1     torch.Size([1024, 256])
rnn.bias_ih_l1       torch.Size([1024])
rnn.bias_hh_l1       torch.Size([1024])
rnn.weight_ih_l2     torch.Size([1024, 256])
rnn.weight_hh_l2     torch.Size([1024, 256])
rnn.bias_ih_l2       torch.Size([1024])
rnn.bias_hh_l2       torch.Size([1024])
fc.weight            torch.Size([83, 256])
fc.bias              torch.Size([83])
Total number of parameters = 1423187 

Number of chars in text : 1,985,223


### Set your training hyperparameters!

In [144]:
net.load_state_dict(torch.load('anna_model.pt'))

<All keys matched successfully>

In [152]:
#val_loss = 1.3498
val_loss = 1.1958
val_loss = 1.2043

In [161]:
batch_size = 10
seq_length = 140
n_epochs =  50 # start small if you are just testing initial behavior

# train the model
val_loss = train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=1000, val_loss_min=val_loss, val_frac=0.1)
print(val_loss)

Validation loss decreases, saving model parameters
Epoch: 1/50... Step: 1000... Loss: 1.1669... Val Loss: 1.1512
Validation loss decreases, saving model parameters
Epoch: 2/50... Step: 2000... Loss: 1.1261... Val Loss: 1.1493
Validation loss decreases, saving model parameters
Epoch: 3/50... Step: 3000... Loss: 1.0658... Val Loss: 1.1486
Epoch: 4/50... Step: 4000... Loss: 1.1972... Val Loss: 1.1492
Epoch: 4/50... Step: 5000... Loss: 1.0790... Val Loss: 1.1487
Epoch: 5/50... Step: 6000... Loss: 1.1821... Val Loss: 1.1486
Validation loss decreases, saving model parameters
Epoch: 6/50... Step: 7000... Loss: 1.1656... Val Loss: 1.1483
Validation loss decreases, saving model parameters
Epoch: 7/50... Step: 8000... Loss: 1.0936... Val Loss: 1.1482
Epoch: 8/50... Step: 9000... Loss: 1.0894... Val Loss: 1.1486
Epoch: 8/50... Step: 10000... Loss: 1.0858... Val Loss: 1.1494
Epoch: 9/50... Step: 11000... Loss: 1.0851... Val Loss: 1.1498
Epoch: 10/50... Step: 12000... Loss: 1.1411... Val Loss: 1.14

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [ ]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


### Show how it is done step by step

In [300]:
x = np.array([[net.char2int['B']]])
x

array([[49]])

In [301]:
x = one_hot_encode(x, net.n_chars)
x

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]]], dtype=float32)

In [302]:
# returns a tuple of hidden state and cell state
h = net.init_hidden(1)
len(h), h[0].shape, h[1].shape

(2, torch.Size([3, 1, 256]), torch.Size([3, 1, 256]))

In [303]:
inp = torch.from_numpy(x).to(device)
inp.is_cuda

True

In [304]:
# hidden state is already on the device where the model is
h[0].is_cuda

True

In [305]:
net.eval()
with torch.no_grad():
    next_char, h = net(inp,h)

In [306]:
next_char

tensor([[-15.6552,   1.6867, -16.9445, -18.4826, -14.7020,  -7.1391,  -4.4100,
          -4.0603, -23.2440, -17.9872,  -7.0180,  -5.2073, -15.6283,  -7.5055,
         -13.5137, -12.7914,   1.4681, -21.9995,   3.5406,  -3.3882,  -2.9762,
          -6.9097,  -9.5095, -18.7235,  -6.0226,  -5.5185,  -7.1476, -10.7954,
           1.0387,  -5.5862, -18.2603, -22.4726, -12.0393, -15.2690,  -9.1073,
         -23.9039, -10.9731, -25.5066, -14.8298,  -3.4448, -11.4609,  -7.7709,
         -19.7498,  -9.3204, -22.0535, -13.9940,   1.6466, -24.8262, -20.0096,
         -14.0743, -12.8643, -23.8616, -23.9243, -25.6489,  -4.0783,  -8.6226,
         -13.1048, -12.1526,  -5.5716,  -4.5847,  -4.6285, -22.0787,  -6.1151,
          -3.0325,  -8.0792,  -8.9869, -15.7782, -14.4186, -10.1173,  -9.4374,
           0.9913,  -3.5431,   0.3915,  -9.6254,  -6.0494, -17.4341,  -6.1099,
         -13.7062, -16.6098,  -9.0038, -12.5257,   3.1387,  -8.4195]],
       device='cuda:0')

In [307]:
next_char = F.softmax(next_char.squeeze_().to('cpu'), dim=0)

In [310]:
p, char = next_char.topk(5)
p, char

(tensor([0.4323, 0.2892, 0.0677, 0.0650, 0.0544]),
 tensor([18, 81,  1, 46, 16]))

In [320]:
next_char = np.random.choice(char.numpy(), p=(p/p.sum()).numpy())
net.int2char[next_char]

'o'

In [174]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        with torch.no_grad():
            out, h = net(inputs, h)

            # get the character probabilities
            p = F.softmax(out, dim=1)
            if(train_on_gpu):
                p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            #print(p, top_ch)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [172]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [182]:
print(sample(net, 200, prime='The ', top_k=10))

The hostess."

"What a patient would tell them with the possible work."

"If I move, honestly you mean out of my father," she said, without her the tribling
room, catching her letter, as though suddenly
so


## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [186]:
loaded = CharRNN(chars, n_hidden, n_layers, drop_prob=0.5)
loaded.load_state_dict(torch.load('anna_model.pt'))

<All keys matched successfully>

In [187]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said. That was some time that he had a children, the day, to say, and still at that instant so well. There has not a person, that
had said the peasants which had to see a means of the country. She had seen the memory of his suffering, but that
she would gaze away, that his
morning was told he was to be talked,
and set there in
the difficulty of the could not stop at once, had not to say that the counting hands arranged an accounts. At the most children who strange this was.

She had to see him the colonel. He could not comprehend, he was all his white thing. She stopped to to be satisfied that she was getting up.

"And that's a fancy to me! If it's
not a contrary."

"Yes, the
princess, I don't know whether it is is such position in the
service of the point, and that inside it's the first man in the window and
herself interesting, but with such a man of the point of horror is a second
and my son to the part, that I have seen him to be a contempt of this solucking at the positi